![An image](https://www.dl.dropboxusercontent.com/s/opt3vuyubou8dgl/demoSlide1.png)
---
#**Wstęp**
`ver. 03.2022`

---
Notatnik jest podzielony na dwie sekcje. W pierwszej sekcji pt. "Synteza mowy - TTS" zaimplementowanych jest kilka otwarto-źródłowych algorytmów syntezy mowy TTS (Text-To-Speech). W drugiej sekcji znajduje się implementacja wybranych algorytmów rozpoznawania mowy z tekstu pisanego STT (Speech-To-Text).

<mark>Szczególnie zalecane jest wykorzystanie przeglądarki Google Chrome</mark>


---


# **Polecenia**


---

1. W sekcji ["Synteza mowy -TTS"](https://colab.research.google.com/drive/13jIPCHHTnZzWkjnuFPRM8fPZlH8Vw-5g#scrollTo=FplH5spO88e1) proszę uruchomić każdy z algorytmów z wymyśloną przez Państwa sekwencją słów i zapisać wynikowe pliki dźwiękowe z syntetyzowaną mową na dysk, a następnie:
  * na podstawie własnych wrażeń słuchowych opisać różnice między algorytmami ([Real-Time Voice Cloning](https://colab.research.google.com/drive/13jIPCHHTnZzWkjnuFPRM8fPZlH8Vw-5g#scrollTo=IhDehA7sT-Gx), [Google TTS (może działać po polsku)](https://colab.research.google.com/drive/13jIPCHHTnZzWkjnuFPRM8fPZlH8Vw-5g#scrollTo=Dp7nPgItKsIb), [Tacotron2 + Waveglow (angielski)](https://colab.research.google.com/drive/13jIPCHHTnZzWkjnuFPRM8fPZlH8Vw-5g#scrollTo=Yl46DA89KETr) i [Mozilla TTS (angielski)](https://colab.research.google.com/drive/13jIPCHHTnZzWkjnuFPRM8fPZlH8Vw-5g#scrollTo=B-e74RzUVqWU)), który z nich jest w stanie bardziej naturalnie syntetyzować ludzką mowę?
2. Proszę nagrać proste zdanie w języku angielskim (np. za pomocą [oprogramowania Audacity](https://manual.audacityteam.org/man/tutorial_your_first_recording.html)), a następnie:
  * dokonać syntezy tego samego zdania wpisując je do algorytmu [Google TTS](https://colab.research.google.com/drive/13jIPCHHTnZzWkjnuFPRM8fPZlH8Vw-5g#scrollTo=Dp7nPgItKsIb) (potrzebna będzie zmiana w kodzie z języka "pl" na "en"), [Tacotron2 + Waveglow](https://colab.research.google.com/drive/13jIPCHHTnZzWkjnuFPRM8fPZlH8Vw-5g#scrollTo=Yl46DA89KETr) oraz [Mozilla TTS](https://colab.research.google.com/drive/13jIPCHHTnZzWkjnuFPRM8fPZlH8Vw-5g#scrollTo=B-e74RzUVqWU) i zapisać wynikowe pliki dźwiękowe na dysk
  * dokonać analizy porównawczej spektrogramów (w [oprogramowaniu Audacity](https://manual.audacityteam.org/man/spectrogram_view.html)) i zapisać wnioski
3. W sekcji ["Rozpoznawanie mowy - STT"](https://colab.research.google.com/drive/13jIPCHHTnZzWkjnuFPRM8fPZlH8Vw-5g#scrollTo=HxWfharV9Mi-) proszę uruchomić każdy z algorytmów po kolei. Plik dźwiękowy może być plikiem wcześniej nagranym przez Was w Audacity (w języku angielskim) albo nagrany za pomocą notatnika (opcja upload/record w podsekcji "Nagranie z mikrofonu lub plik zewnętrzny"). <b>UWAGA:</b> tylko język angielski!
  * zapisać nagrany tekst w postaci pliku dźwiękowego, a w sprawozdaniu zapisać teksty, które udało sie rozpoznać każdemu z algorytmów (CMUSphinx, Google, Mozilla)

**Sprawozdania mają formę dokumentu w dowolnej formie (.pdf, .docx, .odt, ...) oraz dołączonych do sprawozdania plików dźwiękowych z syntetyzowaną mową oraz nagraniem Waszego głosu.**



---


# Wprowadzenie do Google Colab


---
Krótkie wyjaśnienie jak korzystać z notatników w systemie Google Colab:

https://colab.research.google.com/notebooks/basic_features_overview.ipynb


# Ustawienia systemowe
Uzyskanie informacji o wykorzystywanym systemie "w chmurze", na którym będą wykonywane obliczenia.

In [ ]:
!cat /proc/cpuinfo

In [ ]:
!cat /proc/meminfo

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [ ]:
#@title Instalacja bibliotek

%pip install numpy librosa matplotlib

In [ ]:
#@title Zamontowanie przestrzeni dyskowej
#@markdown Będzie potrzebna autoryzacja do dysku Google.
import os 
os.listdir('./')
['.config', 'sample_data', 'drive']
from google.colab import drive
drive.mount('/content/drive')
os.listdir('/content/drive/')

In [ ]:
#@title Import zainstalowanych bibliotek
import soundfile as sf
import io
from six.moves.urllib.request import urlopen

import librosa
import numpy as np
import librosa.display
from matplotlib import pyplot as plt
%matplotlib inline
!apt-get install -y -qq ffmpeg

import IPython
import pandas as pd
import scipy
import sklearn
import os

import seaborn as sns

# Synteza mowy - TTS

## "Real-Time Voice Cloning"

Synteza mowy na podstawie algorytmu opisanego w [pracy dyplomowej](https://matheo.uliege.be/bitstream/2268.2/6801/5/s123578Jemine2019.pdf) oraz [artykle](https://arxiv.org/pdf/1806.04558.pdf). Oryginalne repozytorium z kodem źródłowym znajduje się [tutaj](https://github.com/CorentinJ/Real-Time-Voice-Cloning). 

In [ ]:
#@title Krok 1.Inicjalizacja, instalacja oraz import niezbędnych bibliotek oraz modułów.

#@markdown Uruchomienie za pomocą <code>CTRL+Enter</code> albo ikony <code>play</code>. 
#@markdown Po dwukrotnym kliknięciu w okno tekstowe otworzy się panel edycji kodu źródłowego.

%tensorflow_version 1.x
import os
from os.path import exists, join, basename, splitext

# Skopiowanie repozytorium z kodami źródłowymi 
git_repo_url = 'https://github.com/CorentinJ/Real-Time-Voice-Cloning.git'
project_name = splitext(basename(git_repo_url))[0]
#if not exists(project_name):
if(True):
  # klonowanie repozytorium
  !git clone -q --recursive {git_repo_url}
  # instalacja wymaganych bibliotek
  !cd {project_name} && pip install -q -r requirements.txt
  !pip install -q gdown
  !apt-get install -qq libportaudio2
  !pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip
 
  # Załadowanie wytrenowanych wcześniej modeli sieci
  #!cd {project_name} && wget https://github.com/blue-fish/Real-Time-Voice-Cloning/releases/download/v1.0/pretrained.zip && unzip -o pretrained.zip
  !wget -P {project_name}/encoder/saved_models/ https://www.dl.dropboxusercontent.com/s/uokycp4r9337fik/encoder.pt
  !wget -P {project_name}/synthesizer/saved_models/ https://www.dl.dropboxusercontent.com/s/wwan3xltliow1zv/synthesizer.pt
  !wget -P {project_name}/vocoder/saved_models/ https://www.dl.dropboxusercontent.com/s/4mfqmg2uy7trg8t/vocoder.pt


# Import niezbędnych bibliotek i modułów 
import sys
sys.path.append(project_name)
 
from IPython.display import display, Audio, clear_output
from IPython.utils import io
import ipywidgets as widgets
import numpy as np
from dl_colab_notebooks.audio import record_audio, upload_audio
from scipy.io import wavfile
 
from synthesizer.inference import Synthesizer
from encoder import inference as encoder
from vocoder import inference as vocoder
from pathlib import Path

# Załadowanie odpowiednich modeli sieci
encoder.load_model(project_name / Path("encoder/saved_models/encoder.pt"))
synthesizer = Synthesizer(project_name / Path("synthesizer/saved_models/synthesizer.pt"))
vocoder.load_model(project_name / Path("vocoder/saved_models/vocoder.pt"))

In [ ]:
#@title Krok 2. Nagranie z mikrofonu lub plik zewnętrzny
#@markdown <i>Możliwość nagrania bezpośrednio z mikrofonu lub załadowania pliku
#@markdown z dysku (*.mp3 lub *.wav)</i><br><br>
#@markdown <mark>W przypadku błędów podczas nagrywania z mikrofonu proszę
#@markdown o zmianę źródła nagrywania w pasku adresu przeglądarki (ikona kamerki)
 
SAMPLE_RATE = 44100
record_or_upload = "Record" #@param ["Record", "Upload (.mp3 or .wav)"]
record_seconds =   5#@param {type:"number", min:1, max:10, step:1}
 
embedding = None
def _compute_embedding(audio):
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  wavfile.write('voice_recorded.wav', SAMPLE_RATE, (32767*audio).astype(np.int16))
  global embedding
  embedding = None
  embedding = encoder.embed_utterance(encoder.preprocess_wav(audio, SAMPLE_RATE))
  from google.colab import files
  files.download("voice_recorded.wav")
def _record_audio(b):
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)
def _upload_audio(b):
  clear_output()
  audio = upload_audio(sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)
 
if record_or_upload == "Record":
  button = widgets.Button(description="Record Your Voice")
  button.on_click(_record_audio)
  display(button)
else:
  #button = widgets.Button(description="Upload Voice File")
  #button.on_click(_upload_audio)
  _upload_audio("")

In [ ]:
#@title Krok 3. Synteza tekstu
text = "Raz raz raz raz dwa dwa dwa " #@param {type:"string"}
  
def synthesize(embed, text):
  print("Trwa synteza tekstu do audio...")
  #with io.capture_output() as captured:
  specs = synthesizer.synthesize_spectrograms([text], [embed])
  generated_wav = vocoder.infer_waveform(specs[0])
  generated_wav = np.pad(generated_wav, (0, synthesizer.sample_rate), mode="constant")
  wavfile.write('voice_synth_RTVC.wav', synthesizer.sample_rate, (32767*generated_wav).astype(np.int16))
  from google.colab import files
  files.download("voice_synth_RTVC.wav")
  clear_output()
  display(Audio(generated_wav, rate=synthesizer.sample_rate, autoplay=True))
 
if embedding is None:
  print("first record a voice or upload a voice file!")
else:
  synthesize(embedding, text)

## Google TTS

Synteza mowy na podstawie algorytmu [Google Translate Text-to-Speech](https://cloud.google.com/text-to-speech). Oryginalne repozytorium z kodem źródłowym i portowaniem do Python znajduje się [tutaj](https://gtts.readthedocs.io/en/latest/index.html). 

In [ ]:
#@title Krok 1. Instalacja 
!pip install gTTS
from scipy.io import wavfile

In [ ]:
#@title Krok2. Synteza tekstu
#@markdown Wpisz zdanie do syntezy i język syntezy
text = "Niez\u0142ego strzeli\u0142e\u015B gola. Pi\u0119kny Lolo!" #@param {type:"string"}

from gtts import gTTS
from IPython.display import Audio

lang = 'pl' #@param ["en", "pl"]
tts = gTTS(text,lang=lang)
sound_file = 'voice_synth_TTS.wav'
tts.save('voice_synth_TTS.wav')

from google.colab import files
files.download("voice_synth_TTS.wav")

Audio(sound_file, autoplay=True)

## Tacotron2 + Waveglow

Synteza mowy na podstawie algorytmów [NVIDIA/tacotron2](https://github.com/NVIDIA/tacotron2) oraz [NVIDIA/waveglow](https://github.com/NVIDIA/waveglow). Kody źródłowe na podstawie [tego notatnika](https://colab.research.google.com/github/tugstugi/dl-colab-notebooks/blob/master/notebooks/NVidia_Tacotron2_Waveglow.ipynb).

**WAŻNE.**
Do uruchomienia będzie potrzebne ponowne uruchomienie środowiska wykonawczego >> <code>CTRL + M + .</code> i zakończenie danej sesji >> <code>Menu -> Środowisko wykonawcze -> zarządzaj sesjami -> Zakończ</code>, a następnie uruchomienie kodu w tej sekcji od początku (Krok 1. poniżej).

In [ ]:
#@title Krok 1. Instalacja
#!pip install tensorflow==1.15
%tensorflow_version 1.x
import os
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/NVIDIA/tacotron2.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  # clone and install
  !git clone -q --recursive {git_repo_url}
  !cd {project_name}/waveglow && git checkout 9168aea
  !pip install -q librosa unidecode
  !pip install -q --upgrade gdown
  
import sys
sys.path.append(join(project_name, 'waveglow/'))
sys.path.append(project_name)
import time
import gdown
import matplotlib
import matplotlib.pylab as plt
plt.rcParams["axes.grid"] = False

In [ ]:
#@title Krok 2. Pobieranie wytrenowanych modeli sieci
import gdown
tacotron2_pretrained_model = 'tacotron2_statedict.pt'
if not exists(tacotron2_pretrained_model):
  # download the Tacotron2 pretrained model
  gdown.download(f"https://drive.google.com/uc?export=download&confirm=pbef&id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA",tacotron2_pretrained_model)
  #gdown.download('https://drive.google.com/open?id=1c5ZTuT7J08wLUoVZ2KkUs_VdZuJ86ZqA', tacotron2_pretrained_model, quiet=False)
waveglow_pretrained_model = 'waveglow_old.pt'
if not exists(waveglow_pretrained_model):
  # download the Waveglow pretrained model  
  gdown.download(f"https://drive.google.com/uc?export=download&confirm=pbef&id=1WsibBTsuRg_SF2Z6L6NFRTT-NjEy1oTx",waveglow_pretrained_model)
  #gdown.download('https://drive.google.com/open?id=1WsibBTsuRg_SF2Z6L6NFRTT-NjEy1oTx', waveglow_pretrained_model, quiet=False)

In [ ]:
#@title Krok 3. Inicjalizacja sieci i import bibliotek
#@markdown Kod źródłowy zapożyczony z [tego źródła](https://github.com/NVIDIA/tacotron2/blob/master/inference.ipynb) i uaktualniony do działania w Colab.
import IPython.display as ipd
import numpy as np
import torch

from hparams import create_hparams
from model import Tacotron2
from layers import TacotronSTFT
from audio_processing import griffin_lim
from text import text_to_sequence
from denoiser import Denoiser

def plot_data(data, figsize=(16, 4)):
    fig, axes = plt.subplots(1, len(data), figsize=figsize)
    for i in range(len(data)):
        axes[i].imshow(data[i], aspect='auto', origin='bottom', 
                       interpolation='none', cmap='viridis')

torch.set_grad_enabled(False)
        
# initialize Tacotron2 with the pretrained model
hparams = create_hparams()
hparams.sampling_rate = 22050
model = Tacotron2(hparams)
model.load_state_dict(torch.load(tacotron2_pretrained_model)['state_dict'])
_ = model.cuda().eval()#.half()

# initialize Waveglow with the pretrained model
# waveglow = torch.load(waveglow_pretrained_model)['model']
# WORKAROUND for: https://github.com/NVIDIA/tacotron2/issues/182
import json
from glow import WaveGlow
waveglow_config = json.load(open('%s/waveglow/config.json' % project_name))['waveglow_config']
waveglow = WaveGlow(**waveglow_config)
waveglow.load_state_dict(torch.load(waveglow_pretrained_model)['model'].state_dict())
_ = waveglow.cuda().eval()#.half()
for k in waveglow.convinv:
    k.float()
denoiser = Denoiser(waveglow)


In [ ]:
#@title Krok 4. Synteza tekstu

TEXT = "Basics of Multimedia is awesome!" #@param {type:"string"}

sequence = np.array(text_to_sequence(TEXT, ['english_cleaners']))[None, :]
sequence = torch.autograd.Variable(torch.from_numpy(sequence)).long()
sequence = sequence.cuda()

mel_outputs, mel_outputs_postnet, _, alignments = model.inference(sequence)
#plot_data((mel_outputs.data.cpu().numpy()[0],
#           mel_outputs_postnet.data.cpu().numpy()[0],
#           alignments.data.cpu().numpy()[0].T))

audio = waveglow.infer(mel_outputs_postnet, sigma=0.666)
ipd.Audio(audio[0].data.cpu().numpy(), rate=hparams.sampling_rate)

from scipy.io import wavfile
wavfile.write('voice_synth_T2WaveGlow.wav', hparams.sampling_rate, (32767*audio[0].data.cpu().numpy()).astype(np.int16))
from google.colab import files
files.download("voice_synth_T2WaveGlow.wav")

# remove waveglow bias
audio_denoised = denoiser(audio, strength=0.01)[:, 0]
ipd.Audio(audio_denoised.cpu().numpy(), rate=hparams.sampling_rate)






## Mozilla TTS

Synteza mowy na podstawie algorytmu opisanego w [mozilla/TTS](https://github.com/mozilla/TTS/) oraz dodatkowo z wykjorzystaniem sieci [erogol/WaveRNN](https://github.com/erogol/WaveRNN).

**WAŻNE.**
Do uruchomienia może być potrzebne ponowne uruchomienie środowiska wykonawczego >> <code>CTRL + M + .</code>, a w niektórych przypadkach zakończenie danej sesji >> <code>Środowisko wykonawcze -> zarządzaj sesjami -> Zakończ</code>, a następnie uruchomienie kodu w tej sekcji od początku.


In [ ]:
#@title Krok 1. Instalacja i import bibliotek
import os
import time
from os.path import exists, join, basename, splitext

git_repo_url = 'https://github.com/mozilla/TTS.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  !git clone -q {git_repo_url}
  !cd {project_name} && git checkout Tacotron2-iter-260K-824c091
  !pip install -q --upgrade gdown
  !pip install -q lws librosa Unidecode==0.4.20 tensorboardX git+git://github.com/bootphon/phonemizer@master localimport
  !apt-get install -y espeak
git_repo_url = 'https://github.com/erogol/WaveRNN.git'
project_name = splitext(basename(git_repo_url))[0]
if not exists(project_name):
  !git clone -q {git_repo_url}
  !cd {project_name} && git checkout 8a1c152 && pip install -q -r requirements.txt

  
import sys
sys.path.append('TTS')
sys.path.append('WaveRNN')
!pip install localimport
!pip install gdown
from localimport import localimport
import gdown
  
from IPython.display import Audio, display

In [ ]:
#@title Krok 2. Pobieranie wytrenowanych modeli sieci
# WaveRNN

!mkdir -p wavernn_models tts_models
wavernn_pretrained_model = 'wavernn_models/checkpoint_433000.pth.tar'
if not exists(wavernn_pretrained_model):
  gdown.download(f"https://drive.google.com/uc?export=download&confirm=pbef&id=12GRFk5mcTDXqAdO5mR81E-DpTk8v2YS9",wavernn_pretrained_model)
  #gdown.download('https://drive.google.com/uc?id=12GRFk5mcTDXqAdO5mR81E-DpTk8v2YS9', wavernn_pretrained_model, quiet=False)
wavernn_pretrained_model_config = 'wavernn_models/config.json'
if not exists(wavernn_pretrained_model_config):
  gdown.download(f"https://drive.google.com/uc?export=download&confirm=pbef&id=1kiAGjq83wM3POG736GoyWOOcqwXhBulv",wavernn_pretrained_model_config)
  #gdown.download('https://drive.google.com/uc?id=1kiAGjq83wM3POG736GoyWOOcqwXhBulv', wavernn_pretrained_model_config, quiet=False)
    
# TTS
tts_pretrained_model = 'tts_models/checkpoint_261000.pth.tar'
if not exists(tts_pretrained_model):
  gdown.download(f"https://drive.google.com/uc?export=download&confirm=pbef&id=1otOqpixEsHf7SbOZIcttv3O7pG0EadDx",tts_pretrained_model)
  #gdown.download(f'https://drive.google.com/uc?id=1otOqpixEsHf7SbOZIcttv3O7pG0EadDx', tts_pretrained_model, quiet=False)
tts_pretrained_model_config = 'tts_models/config.json'
if not exists(tts_pretrained_model_config):
  gdown.download(f"https://drive.google.com/uc?export=download&confirm=pbef&id=1IJaGo0BdMQjbnCcOL4fPOieOEWMOsXE-",tts_pretrained_model_config)
  #gdown.download('https://drive.google.com/uc?id=1IJaGo0BdMQjbnCcOL4fPOieOEWMOsXE-', tts_pretrained_model_config, quiet=False)

In [ ]:
#@title Krok 3. Inicjalizacja sieci
#@markdown Kod źródłowy zapożyczony [ze źródła](https://github.com/tugstugi/dl-colab-notebooks/blob/master/notebooks/Mozilla_TTS_WaveRNN.ipynb).
import io
import torch 
import time
import numpy as np
from collections import OrderedDict
from matplotlib import pylab as plt
import IPython

%pylab inline
rcParams["figure.figsize"] = (16,5)

import librosa
import librosa.display

!pip install phonemizer
!pip install unidecode

from TTS.models.tacotron import Tacotron 
from TTS.layers import *
from TTS.utils.data import *
from TTS.utils.audio import AudioProcessor
from TTS.utils.generic_utils import load_config, setup_model
from TTS.utils.text import text_to_sequence
from TTS.utils.synthesis import synthesis
from TTS.utils.visual import visualize

def tts(model, text, CONFIG, use_cuda, ap, use_gl, speaker_id=None, figures=True):
    t_1 = time.time()
    waveform, alignment, mel_spec, mel_postnet_spec, stop_tokens = synthesis(model, text, CONFIG, use_cuda, ap, truncated=True, enable_eos_bos_chars=CONFIG.enable_eos_bos_chars)
    if CONFIG.model == "Tacotron" and not use_gl:
        mel_postnet_spec = ap.out_linear_to_mel(mel_postnet_spec.T).T
    if not use_gl:
        waveform = wavernn.generate(torch.FloatTensor(mel_postnet_spec.T).unsqueeze(0).cuda(), batched=batched_wavernn, target=11000, overlap=550)

    print(" >  Run-time: {}".format(time.time() - t_1))
    if figures:                                                                                                         
        visualize(alignment, mel_postnet_spec, stop_tokens, text, ap.hop_length, CONFIG, mel_spec)                                                                       
    IPython.display.display(Audio(waveform, rate=CONFIG.audio['sample_rate']))  
    #os.makedirs(OUT_FOLDER, exist_ok=True)
    #file_name = text.replace(" ", "_").replace(".","") + ".wav"
    #out_path = os.path.join(OUT_FOLDER, file_name)
    #ap.save_wav(waveform, out_path)
    return alignment, mel_postnet_spec, stop_tokens, waveform
  
use_cuda = True
batched_wavernn = True

# initialize TTS
CONFIG = load_config(tts_pretrained_model_config)
from TTS.utils.text.symbols import symbols, phonemes
# load the model
num_chars = len(phonemes) if CONFIG.use_phonemes else len(symbols)
model = setup_model(num_chars, CONFIG)
# load the audio processor
ap = AudioProcessor(**CONFIG.audio)         
# load model state
if use_cuda:
    cp = torch.load(tts_pretrained_model)
else:
    cp = torch.load(tts_pretrained_model, map_location=lambda storage, loc: storage)

# load the model
model.load_state_dict(cp['model'])
if use_cuda:
    model.cuda()
model.eval()
print(cp['step'])
model.decoder.max_decoder_steps = 2000

# initialize WaveRNN
VOCODER_CONFIG = load_config(wavernn_pretrained_model_config)
with localimport('/content/WaveRNN') as _importer:
  from models.wavernn import Model
bits = 10

wavernn = Model(
        rnn_dims=512,
        fc_dims=512,
        mode="mold",
        pad=2,
        upsample_factors=VOCODER_CONFIG.upsample_factors,  # set this depending on dataset
        feat_dims=VOCODER_CONFIG.audio["num_mels"],
        compute_dims=128,
        res_out_dims=128,
        res_blocks=10,
        hop_length=ap.hop_length,
        sample_rate=ap.sample_rate,
    ).cuda()
check = torch.load(wavernn_pretrained_model)
wavernn.load_state_dict(check['model'])
if use_cuda:
    wavernn.cuda()
wavernn.eval()
print(check['step'])

In [ ]:
#@title Krok 4. Synteza tekstu

SENTENCE = 'Oh my, my, my, the spring is coming and I feel Pola Mokotowskie very clear' #@param {type:"string"}
align, spec, stop_tokens, wav = tts(model, SENTENCE, CONFIG, use_cuda, ap, speaker_id=0, use_gl=False, figures=False)

from scipy.io import wavfile
wavfile.write('voice_synth_MTTSWaveRNN.wav', 22050, (32767*wav).astype(np.int16))
from google.colab import files
files.download("voice_synth_MTTSWaveRNN.wav")


#Rozpoznawanie mowy - STT

## CMU Sphinx i Google STT

### CMU Sphinx

Biblioteka [CMUSphinx](https://cmusphinx.github.io/), która zawiera wiele alogorytmów i rozwiązań przeznaczonych do automatycznego rozpoznawania mowy. Biblioteka [PocketSphinx](https://github.com/cmusphinx/pocketsphinx-python), która implementuje wybrane algorytmy w jęzku Python.

### Google Speech-To-Text

Biblioteka [Google](https://cloud.google.com/speech-to-text).

Uruchomienie obydwu ww. bibliotek jest możliwe tutaj dzięki integracji w narzędziu [SpeechRecognition](https://github.com/Uberi/speech_recognition).

<br>

**WAŻNE.**
Do uruchomienia może być potrzebne ponowne uruchomienie środowiska wykonawczego >> <code>CTRL + M + .</code>, a w niektórych przypadkach zakończenie danej sesji >> <code>Środowisko wykonawcze -> zarządzaj sesjami -> Zakończ</code>, a następnie uruchomienie kodu w tej sekcji od początku.

In [ ]:
#@title Krok 1. Instalacja niezbędnych bibliotek i modułów
#@markdown Dla ciekawych - dwukrotne kliknięcie w nagłówek otworzy okno z kodem źródłowym.

# Instalacja CMUSphinx
!apt-get install -y swig libpulse-dev
!swig -version
!pip3 install pocketsphinx
!pip3 list | grep pocketsphinx

# Instalacja pakietu SpeechRecognition
!pip3 install SpeechRecognition
import os

# Instalacja Mozilla DeepSpeech
!pip install deepspeech==0.6.0
if (os.path.isfile("deepspeech-0.6.0-models.tar.gz")==False):
  !curl -LO https://github.com/mozilla/DeepSpeech/releases/download/v0.6.0/deepspeech-0.6.0-models.tar.gz
!tar -xvzf deepspeech-0.6.0-models.tar.gz
!ls -l ./deepspeech-0.6.0-models/
#!pip3 install --upgrade deepspeech

# Instalacja
!pip install librosa
!apt-get install -y -qq ffmpeg

# Inicjalizacja biblioteki
import pocketsphinx

 
MODELDIR = os.path.join(os.path.dirname(pocketsphinx.__file__), 'model')
 
config = pocketsphinx.Decoder.default_config()
config.set_string('-hmm', os.path.join(MODELDIR, 'en-us'))
config.set_string('-lm', os.path.join(MODELDIR, 'en-us.lm.bin'))
config.set_string('-dict', os.path.join(MODELDIR, 'cmudict-en-us.dict'))
 
decoder = pocketsphinx.Decoder(config)

In [ ]:
#@title Krok 2. Nagranie z mikrofonu lub plik zewnętrzny
#@markdown <i>Możliwość nagrania bezpośrednio z mikrofonu lub załadowania pliku
#@markdown z dysku (*.mp3 lub *.wav)</i><br><br>
#@markdown <b>WAŻNE!</b> nagranie lub plik to zdanie w języku angielskim <br><br>
#@markdown Dla ciekawych - dwukrotne kliknięcie w nagłówek otworzy okno z kodem źródłowym.

# Import i instalacja niezbędnych bibliotek i modułów do nagrywania i odtwarzania
!apt-get install -qq libportaudio2
!pip install -q https://github.com/tugstugi/dl-colab-notebooks/archive/colab_utils.zip

import sys
 
from IPython.display import display, Audio, clear_output
from IPython.utils import io
import ipywidgets as widgets
import numpy as np
from dl_colab_notebooks.audio import record_audio, upload_audio
from scipy.io import wavfile
 
SAMPLE_RATE = 44100
record_or_upload = "Record" #@param ["Record", "Upload (.mp3 or .wav)"]
record_seconds =   10#@param {type:"number", min:1, max:10, step:1}
 
embedding = None
def _compute_embedding(audio):
  display(Audio(audio, rate=SAMPLE_RATE, autoplay=True))
  wavfile.write('voice_recorded_uploaded.wav', SAMPLE_RATE, (32767*audio).astype(np.int16))
  global embedding
  from google.colab import files
  files.download("voice_recorded_uploaded.wav")
def _record_audio(b):
  clear_output()
  audio = record_audio(record_seconds, sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)
def _upload_audio(b):
  clear_output()
  audio = upload_audio(sample_rate=SAMPLE_RATE)
  _compute_embedding(audio)
 
if record_or_upload == "Record":
  button = widgets.Button(description="Record Your Voice")
  button.on_click(_record_audio)
  display(button)
else:
  #button = widgets.Button(description="Upload Voice File")
  #button.on_click(_upload_audio)
  _upload_audio("")

In [ ]:
#@title Krok 3. SpeechRecognition
#@markdown <i>Rozpoznawanie za pomocą algorytmu CMUSphinx i Google wcześniej nagranego lub
#@markdown wgranego pliku</i><br><br>
#@markdown Dla ciekawych - dwukrotne kliknięcie w nagłówek otworzy okno z kodem źródłowym.

import speech_recognition as sr
from enum import Enum, unique
 
@unique
class ASREngine(Enum):
    sphinx = 0
    google = 1
 
def speech_to_text(filename: str, engine: ASREngine, language: str, show_all: bool = False) -> str:
    r = sr.Recognizer()

    try:
      with sr.AudioFile(filename) as source:
          audio = r.record(source)
 
      asr_functions = {
          ASREngine.sphinx: r.recognize_sphinx,
          ASREngine.google: r.recognize_google,
      }
 
      response = asr_functions[engine](audio, language=language, show_all=show_all)
      return response
    except:
      print('Brak pliku audio\nproszę wcześniej nagrać lub załadować plik')
 
filename = 'voice_recorded_uploaded.wav'
lang = 'en-US'
 
for asr_engine in ASREngine:
  try:
    response = speech_to_text(filename, asr_engine, language=lang)
    print('{0}: "{1}"'.format(asr_engine.name, response))
  except sr.UnknownValueError:
    print('{0} could not understand audio'.format(asr_engine.name))
  except sr.RequestError as e:
    print('{0} error: {0}'.format(asr_engine.name, e))

## Mozilla DeepSpeech

Algorytm [Mozilla DeepSpeech](https://arxiv.org/abs/1412.5567) z kodem źródłowym [w tym miejscu](https://github.com/mozilla/DeepSpeech).

In [ ]:
#@title Krok 1. Uruchomienie DeepSpeech
#@markdown <i>Rozpoznawanie za pomocą algorytmu Mozilla DeepSpeech wcześniej nagranego lub
#@markdown wgranego pliku</i><br><br>
#@markdown Dla ciekawych - dwukrotne kliknięcie w nagłówek otworzy okno z kodem źródłowym.
import numpy as np
import deepspeech

model_file_path = 'deepspeech-0.6.0-models/output_graph.pbmm'
beam_width = 500
model = deepspeech.Model(model_file_path, beam_width)

# Add language model for better accuracy
lm_file_path = 'deepspeech-0.6.0-models/lm.binary'
trie_file_path = 'deepspeech-0.6.0-models/trie'
lm_alpha = 0.75
lm_beta = 1.85
model.enableDecoderWithLM(lm_file_path, trie_file_path, lm_alpha, lm_beta)

import numpy as np
import wave 

def deepspeech_batch_stt(filename: str) -> str:
    import librosa    
    import soundfile as sf

    # Get example audio file
    data, samplerate = sf.read(filename, dtype='float32')
    data = data.T
    data16kfloat = librosa.resample(data, samplerate, 16000)
    sf.write('voice16.wav', data16kfloat, 16000, subtype='PCM_16')
    data16, samplerate = sf.read('voice16.wav', dtype='int16')
    #return model.stt(data16)
    return model.stt(data16)

filename = 'voice_recorded_uploaded.wav'
print('WYNIK: "{}"'.format(deepspeech_batch_stt(filename)))